# 5 minute append to state column 

After taking some time to look around I found usaddress for parsing states.  This notebook shows how easy it is to parse addresses with usaddress.  I feel that the list of tuples is a bit clunky, but it works and I only spotted one error in the results... Indianna??  really??

In [2]:
import pandas as pd
import seaborn as sns
import usaddress
% matplotlib inline

## Load Data

In [3]:
df = pd.read_csv('https://query.data.world/s/78ou6jcu4jfseul53lu1w3nio')

In [4]:
df.head()

,location,crowd-low-estimate,crowd-high-estimate,mean-high-low,source,Latitude,Longitude
0,"Abilene, TX",200,200,200.0,http://www.reporternews.com/story/news/local/2...,32.576489,-99.665323
1,"Accident, MD",54,54,54.0,Twitter; on-site witness,39.628700,-79.319760
2,"Adak, AK",10,10,10.0,adn.com,0.000000,0.000000
3,"Adrian, MI",130,150,140.0,https://www.facebook.com/events/847360115406578/,41.889943,-84.065892
4,"Ajo, AZ",250,250,250.0,https://www.facebook.com/plugins/post.php?href...,32.384890,-112.890110


## original method
using last 2 characters of location

In [5]:
df['State'] = df['location'].str[-2:]

In [6]:
df.groupby('State').count().sort_values('location', ascending=False)[['location']].head()

,location
State,
CA,69
WA,27
NY,25
OR,24
AK,22


Looks Like there are a few errors in State that would need cleansed (at least 12)

15 minutes is not enough to fix this by hand, I know there is a package that would do this faster that has been mentioned on talk python to me.  Ill have to look into this package another day

In [7]:
len(df.groupby('State').count())

62

In [8]:
df.groupby('State').count().sort_values('location', ascending=False)[['location']].tail(15)

,location
State,
DE,2
OK,2
LA,2
DC,1
RI,1
ah,1
er,1
es,1
ge,1


## Usaddress Method

In [9]:
df['Address'] = df['location'].apply(usaddress.parse)

In [10]:
df['Address'].iloc[10:20]

10         [(Albuquerque,, PlaceName), (NM, StateName)]
11    [(Almanor, PlaceName), (West,, PlaceName), (CA...
12              [(Alpine,, PlaceName), (TX, StateName)]
13            [(Amarillo,, PlaceName), (TX, StateName)]
14    [(Amelia, StreetName), (Island,, StreetNamePos...
15           [(Anacortes,, PlaceName), (WA, StateName)]
16           [(Anchorage,, PlaceName), (AK, StateName)]
17    [(Ann, PlaceName), (Arbor,, PlaceName), (MI, S...
18           [(Annapolis,, PlaceName), (MD, StateName)]
19           [(Arlington,, PlaceName), (VA, StateName)]
Name: Address, dtype: object

In [11]:
pd.DataFrame.from_records(df['Address'].values.tolist()).iloc[10:20]

,0,1,2,3,4,5
10,"(Albuquerque,, PlaceName)","(NM, StateName)",None,None,None,None
11,"(Almanor, PlaceName)","(West,, PlaceName)","(CA, StateName)",None,None,None
12,"(Alpine,, PlaceName)","(TX, StateName)",None,None,None,None
13,"(Amarillo,, PlaceName)","(TX, StateName)",None,None,None,None
14,"(Amelia, StreetName)","(Island,, StreetNamePostType)","(FL, OccupancyType)",None,None,None
15,"(Anacortes,, PlaceName)","(WA, StateName)",None,None,None,None
16,"(Anchorage,, PlaceName)","(AK, StateName)",None,None,None,None
17,"(Ann, PlaceName)","(Arbor,, PlaceName)","(MI, StateName)",None,None,None
18,"(Annapolis,, PlaceName)","(MD, StateName)",None,None,None,None
19,"(Arlington,, PlaceName)","(VA, StateName)",None,None,None,None


In [12]:
def get_state(lst):
    for tpl in lst:
        if tpl[1] == 'StateName':
            return tpl[0]
    return None

In [13]:
df['State'] = df['Address'].apply(get_state)

In [14]:
len(df.groupby('State').count())

53

# Errors? 53?
Looks like there were 53 states it counted PR (Puerto Rico), and DC which is good. But it also included Indianna?? not sure why, but I am impressed with the reults.

In [15]:
df.groupby('State').count().sort_values('location', ascending=False)[['location']]

,location
State,
CA,70
WA,27
NY,25
OR,24
AK,22
MI,20
TX,20
PA,19
CO,19


In [22]:
df['Address'].values.tolist()

[('Amarillo,', 'PlaceName'), ('TX', 'StateName')]

In [24]:
!git add .
!git commit -m "added 5min append State exporing the usaddress package"
!git push

The file will have its original line endings in your working directory.
The file will have its original line endings in your working directory.


[master c370888] added 5min append State exporing the usaddress package
 Committer: Rhiannon Walker <Rhiannon Walker>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly. Run the
following command and follow the instructions in your editor to edit
your configuration file:

    git config --global --edit

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 2 files changed, 1998 insertions(+)
 create mode 100644 .ipynb_checkpoints/5min append State-checkpoint.ipynb
 create mode 100644 5min append State.ipynb


To https://github.com/WaylonWalker/15min_WomensMarch.git
 ! [rejected]        master -> master (fetch first)
error: failed to push some refs to 'https://github.com/WaylonWalker/15min_WomensMarch.git'
hint: Updates were rejected because the remote contains work that you do
hint: not have locally. This is usually caused by another repository pushing
hint: to the same ref. You may want to first integrate the remote changes
hint: (e.g., 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


In [25]:
!git pull

Merge made by the 'recursive' strategy.
 README.md | 8 ++++++++
 1 file changed, 8 insertions(+)
 create mode 100644 README.md


From https://github.com/WaylonWalker/15min_WomensMarch
   680e352..291d5f8  master     -> origin/master
